# Example of RO-Crates ingestion and fuseki SparQL
- set up local SparQL DB
- get several RO-Crates published
- create a graph and add it to the SparQL

This NB should mirror the [tutorial](https://py-ualg.github.io/biohap/biodata_pt/training_2511.html) working directly on `fuseki`

**Fuseki setup**

Fuseki is a java SparQL endpoint server.

Get java
```
sudo apt update
sudo apt install -y openjdk-17-jre-headless
java -version
```

- download the zip file from https://jena.apache.org/download/index.cgi
- run the fuseki server script
```
cd apache-jena-fuseki-5.6.0/
./fuseki-server
```
the server is at
http://localhost:3030/#/

**Preparation of a dataset**

- I want to keep this in pure python
- use `rdflib rdflib-jsonld requests`

## 0. Imports and functions defs

In [1]:
import os
import re
import json
import requests

import pandas as pd

from pathlib import Path
from rdflib import Graph, Namespace, URIRef, RDF
from urllib.parse import urlparse
from utils import sparql_json_to_df, jsonld_to_rdflib

pd.options.display.max_columns = None
pd.set_option("display.max_colwidth", None)

### Methods

In [2]:
def fuseki_create_dataset(name: str) -> None:
    """Create a new in-memory dataset in a local Apache Jena Fuseki server.
    Args:
        name (str): Name of the dataset to create.
    
    Returns:
        None
    """
    fuseki_admin_url = "http://localhost:3030/$/datasets"
    # Form data
    form_data = {
        "dbName": name,   # dataset name
        "dbType": "mem"       # in-memory
    }

    # Headers to enforce form encoding
    headers = {
        "Content-Type": "application/x-www-form-urlencoded"
    }

    # POST request
    resp = requests.post(fuseki_admin_url, data=form_data, headers=headers)
    if resp.status_code != 200:
        print("Dataset creation failed for", name, resp.status_code)
        print("Server response:", resp.text[:1000])
    else:
        print("Dataset created:", name)


def create_upload_ds(name: str, contents: bytes) -> None:
    """
    Create a new dataset in Fuseki and upload contents to it.

    Args:
        name (str): Name of the dataset.
        contents (bytes): RDF data in Turtle format.
    
    Returns:
        None
    """
    fuseki_create_dataset(name)

    # Now upload the data to the named graph
    gsp_endpoint = f"http://localhost:3030/{name}/data"
    headers = {"Content-Type": "text/turtle"}     # sending Turtle
    resp = requests.post(
        gsp_endpoint,
        data=contents,
        headers=headers,
        timeout=60,
    )
    if resp.status_code != 200:
        print("Upload failed for dataset", name, resp.status_code)
        print("Server response:", resp.text[:1000])
    else:
        print("Upload succeeded for dataset", name)

### 0.5 Get current EMO-BON RO-Crates
- easiest is to clone the `https://github.com/emo-bon/analysis-results-cluster-01-crate` repo
- semi-hardcoded option to get data from GitHub can be found in `03_fuseki_emobon_GH.ipynb`
- TODO: rewrite in existing fuseki engines such pyfuseki, https://yubincloud.github.io/pyfuseki/


## 1. Serialize `ro-crate-metadata.json` to `.ttl`

In [ ]:
# USER SETTINGS
rocrate_folder = Path.home() / "coding/ro-crates/analysis-results-cluster-01-crate/"
process_N = 10  # number of crates to process

In [4]:

# loop over all ro-crates in the folder
count = 0
for crate_path in rocrate_folder.glob("*/"):
    if 'ro-crate-metadata.json' not in os.listdir(crate_path):
        print(f"Skipping {crate_path}, no ro-crate-metadata.json found.")
        continue

    print(f"Processing crate at {crate_path}...")
    with open(crate_path / "ro-crate-metadata.json", "r") as f:
        metadata = json.load(f)
    graph = jsonld_to_rdflib(metadata)

    # Serialize to TTL format and save to file
    ttl_content = graph.serialize(format='turtle')

    # print(crate_path.parent / (crate_path.name + '.ttl'))
    with open(crate_path.parent / (crate_path.name + '.ttl'), 'w', encoding='utf-8') as f:
        f.write(ttl_content)

    count += 1
    if count >= process_N:
        break


Processing crate at /home/david-palecek/coding/ro-crates/analysis-results-cluster-01-crate/EMOBON_NRMCB_So_12-ro-crate...
Processing crate at /home/david-palecek/coding/ro-crates/analysis-results-cluster-01-crate/EMOBON_RFormosa_So_6-ro-crate...
Skipping /home/david-palecek/coding/ro-crates/analysis-results-cluster-01-crate/.github, no ro-crate-metadata.json found.
Processing crate at /home/david-palecek/coding/ro-crates/analysis-results-cluster-01-crate/EMOBON_PiEGetxo_Wa_14-ro-crate...
Processing crate at /home/david-palecek/coding/ro-crates/analysis-results-cluster-01-crate/EMOBON_BPNS_So_5-ro-crate...
Processing crate at /home/david-palecek/coding/ro-crates/analysis-results-cluster-01-crate/EMOBON_OSD74_Wa_12-ro-crate...
Skipping /home/david-palecek/coding/ro-crates/analysis-results-cluster-01-crate/.git, no ro-crate-metadata.json found.
Processing crate at /home/david-palecek/coding/ro-crates/analysis-results-cluster-01-crate/EMOBON_ROSKOGO_Wa_27-ro-crate...
Processing crate at /h

## 2. Repeat queries but from python
- you should have running `fuseki` on the `localhost:3030`
- it should be manually populated with EMO-BON RO-Crates with dataset name `emobon`

### How many triples do we have?

In [ ]:
q = """
SELECT (COUNT(*) AS ?c)
WHERE { 
    ?s ?p ?o
}
"""
r = requests.get("http://localhost:3030/emobon", params={"query": q}, headers={"Accept": "application/sparql-results+json"})
print(r.json())

In [ ]:
df = sparql_json_to_df(r.json())
print(df)

### Filter our all the `text/html` files

In [ ]:
q = """
PREFIX sdo: <http://schema.org/>

SELECT ?x ?dtype
WHERE {
  ?x sdo:encodingFormat ?dtype .
  FILTER regex(str(?dtype), "^text/html", "i")
}
"""
r = requests.get("http://localhost:3030/emobon", params={"query": q}, headers={"Accept": "application/sparql-results+json"})
df = sparql_json_to_df(r.json())
df

### Return also the `sdo:downloadUrl` of those files.

In [ ]:
q = """
PREFIX sdo: <http://schema.org/>

SELECT ?x ?dtype ?durl
WHERE {
  ?x sdo:encodingFormat ?dtype ;
     sdo:downloadUrl ?durl .
  FILTER regex(str(?dtype), "^text/html", "i")
}
"""
r = requests.get("http://localhost:3030/emobon", params={"query": q}, headers={"Accept": "application/sparql-results+json"})
df = sparql_json_to_df(r.json())
df

### Return SSU taxonomy download links

In [ ]:
q = """
PREFIX sdo: <http://schema.org/>

SELECT ?subject ?predicate ?object ?durl
WHERE {
  ?subject ?predicate ?object .
  FILTER regex(str(?object), "SSU-taxonomy-summary", "i")
  OPTIONAL { ?object sdo:downloadUrl ?durl }
}
LIMIT 50
"""

r = requests.get(
    "http://localhost:3030/emobon",
    params={"query": q},
    headers={"Accept": "application/sparql-results+json"},
)

df = sparql_json_to_df(r.json())
df

### SSU taxonomy display
- from the object values we see the taxonomy is in ttl format, which means it has been triplicated during so called `semantic uplift`

In [ ]:
url = df["durl"].dropna().unique()[0]
r = requests.get(url)
# save to a file
with open("ssu_example.ttl", "wb") as f:
    f.write(r.content)

In [ ]:
url

In [ ]:
g = Graph()
g.parse("ssu_example.ttl", format="turtle")

# Convert all triples into a list of tuples
triples = [(str(s), str(p), str(o)) for s, p, o in g]

# Make a pandas DataFrame
df = pd.DataFrame(triples, columns=["subject", "predicate", "object"])
df.head(10)

### Translate SSU triples into a taxonomy DF

In [ ]:
COLUMNS = [
    'ncbi_tax_id', 'abundance', 'superkingdom',
    'kingdom', 'phylum', 'class', 'order',
    'family', 'genus', 'species'
]
RANKS = ['superkingdom','kingdom','phylum','class','order','family','genus','species']

# ----- helper functions -----------------------------------------------------
def as_str(node):
    return None if node is None else str(node)

def extract_ncbi_id(uri_str):
    """Extract numeric taxid from NCBI URI or fragment like #41873."""
    if not uri_str:
        return None
    m = re.search(r'[?&]id=(\d+)', uri_str)
    if m:
        return m.group(1)
    m = re.search(r'#(\d+)$', uri_str)
    if m:
        return m.group(1)
    m = re.search(r'/(\d+)(?:$|[/?#])', uri_str)
    if m:
        return m.group(1)
    return None

def last_path_segment(uri_str):
    if not uri_str:
        return None
    p = urlparse(uri_str)
    if p.fragment:
        return p.fragment
    seg = p.path.rstrip('/').split('/')[-1]
    return seg if seg != '' else None


In [ ]:
# ----- load graph -----------------------------------------------------------
g = Graph()
g.parse("ssu_example.ttl", format="turtle")   # <-- change path if needed

# ----- namespaces used in your sample --------------------------------------
PROD = Namespace("https://data.emobon.embrc.eu/ns/product#")
DCT  = Namespace("http://purl.org/dc/terms/")
SCHEMA = Namespace("http://schema.org/")  # not required here but safe

# ----- accumulate rows keyed by (source_material_ID, ncbi_tax_id) -----------
acc = {}  # (source_id, taxid) -> dict of columns

def ensure_row(source_id, taxid):
    key = (source_id or "unknown_sample", taxid or "unknown_taxid")
    if key not in acc:
        acc[key] = {col: None for col in COLUMNS}
        acc[key]['ncbi_tax_id'] = taxid
    return acc[key]

# ----- iterate TaxonomicAnnotation nodes -----------------------------------
for ta_node in g.subjects(RDF.type, PROD.TaxonomicAnnotation):
    # read annotation-level properties
    sample_uri = g.value(ta_node, PROD.ofSample) or g.value(ta_node, DCT.ofSample)
    source_id = last_path_segment(as_str(sample_uri))
    identifier = g.value(ta_node, DCT.identifier)  # expected to be NCBI URI
    identifier_s = as_str(identifier)
    ncbi_id = extract_ncbi_id(identifier_s)

    # abundance (prod:ssuRNA) and otuID
    abundance_term = g.value(ta_node, PROD.ssuRNA)
    # sometimes abundance literal typed - convert to int if possible
    try:
        abundance = int(str(abundance_term))
    except Exception:
        abundance = str(abundance_term)

    # create/ensure row
    row = ensure_row(source_id, ncbi_id)
    row['abundance'] = abundance

    # now look up the taxon node (identifier) and extract rank / scientificName
    if identifier is not None:
        tax_subject = URIRef(identifier_s)
        sci_name_term = g.value(tax_subject, DCT.scientificName) or g.value(tax_subject, DCT.title)
        rank_term = g.value(tax_subject, DCT.taxonRank)
        sci_name = as_str(sci_name_term)
        rank = as_str(rank_term).lower() if rank_term is not None else None

        if rank and sci_name:
            # if rank is one of our expected ranks, store at that column
            if rank in RANKS:
                row[rank] = sci_name
        else:
            raise ValueError(f"Missing rank or scientific name for taxon {identifier_s}")

# ----- build final DataFrame ------------------------------------------------
out_rows = []
for (source_id, taxid), vals in acc.items():
    # enforce ncbi_tax_id numeric when possible
    try:
        vals['ncbi_tax_id'] = int(taxid) if (taxid and taxid != "unknown_taxid") else None
    except Exception:
        vals['ncbi_tax_id'] = taxid
    vals['source_material_ID'] = source_id
    out_rows.append(vals)

df = pd.DataFrame(out_rows)

# set index like your example and reorder columns
df['source_material_ID'] = df['source_material_ID'].fillna('unknown_sample')
df['ncbi_tax_id'] = df['ncbi_tax_id'].fillna('unknown_taxid')

df = df.set_index(['source_material_ID', 'ncbi_tax_id'])
final_cols = ['abundance'] + RANKS
# ensure columns exist in the DataFrame before slicing
final_cols = [c for c in final_cols if c in df.columns]
df = df[final_cols].sort_index()

# convert abundance to numeric (if any)
if 'abundance' in df.columns:
    df['abundance'] = pd.to_numeric(df['abundance'], errors='coerce')

print(df.shape)
df.head(20)

## 3. Create dataset if not yet on fuseki

In [4]:
# check if dataset exists, if not create it
fuseki_datasets_url = "http://localhost:3030/$/datasets"
dataset = "emobon_python"   # if does not exist yet, it will be created

# get existing datasets
resp = requests.get(fuseki_datasets_url)
resp.raise_for_status()
datasets_info = resp.json()

existing_datasets = [ds["ds.name"] for ds in datasets_info["datasets"]]

print("Existing datasets:", existing_datasets)
if dataset not in existing_datasets:
    fuseki_create_dataset(dataset)

# upload TTL content to the dataset
fuseki_url = f"http://localhost:3030/{dataset}/data"
headers = {"Content-Type": "text/turtle"}


Existing datasets: []
Dataset created: emobon_python


In [5]:
# ingest one ro-crate as example
with open(rocrate_folder / "EMOBON_OOB_So_26-ro-crate.ttl", "r") as f:
    ttl_content = f.read()
    
resp = requests.put(fuseki_url, data=ttl_content.encode("utf-8"), headers=headers, timeout=60)

# raise for HTTP error
try:
    resp.raise_for_status()
except requests.HTTPError as e:
    raise RuntimeError(f"Upload failed: {resp.status_code} {resp.text}") from e

## 4. Ingestion to fuseki

### Loop over `ttl`s in a folder

In [6]:
for filename in os.listdir(rocrate_folder):
    if not filename.endswith("-ro-crate.ttl"):
        continue

    path = os.path.join(rocrate_folder, filename)
    print("Uploading (append):", filename)
    with open(path, "rb") as f:
        ttl_bytes = f.read()

    create_upload_ds(dataset, ttl_bytes)

print("All files uploaded (appended).")

Uploading (append): EMOBON_RFormosa_So_11-ro-crate.ttl
Dataset creation failed for emobon_python 409
Server response: Name already registered '/emobon_python'

Upload succeeded for dataset emobon_python
Uploading (append): EMOBON_OSD74_Wa_12-ro-crate.ttl
Dataset creation failed for emobon_python 409
Server response: Name already registered '/emobon_python'

Upload succeeded for dataset emobon_python
Uploading (append): EMOBON_NRMCB_So_12-ro-crate.ttl
Dataset creation failed for emobon_python 409
Server response: Name already registered '/emobon_python'

Upload succeeded for dataset emobon_python
Uploading (append): EMOBON_VB_Wa_94-ro-crate.ttl
Dataset creation failed for emobon_python 409
Server response: Name already registered '/emobon_python'

Upload succeeded for dataset emobon_python
Uploading (append): EMOBON_NRMCB_Wa_69-ro-crate.ttl
Dataset creation failed for emobon_python 409
Server response: Name already registered '/emobon_python'

Upload succeeded for dataset emobon_python


### sidetrack to `DVC`
- I want to get the ttl file from the NB, not putting the `sdo:downloadUrl` into the browser

In [7]:
os.environ['AWS_NO_SIGN_REQUEST'] = '1'
os.environ.pop('AWS_PROFILE', None)   # avoid using a missing profile

import dvc.api
from dvc.api import DVCFileSystem
import boto3
from botocore import UNSIGNED
from botocore.client import Config


In [8]:
def get_single_file_s3(repo_folder: str, path: str) -> tuple[str, bytes]:
    """
    Get a single file from an S3 bucket using DVC and boto3 without credentials.
    Args:
        repo_folder (str): Path to the DVC repository.
        path (str): Path to the file within the DVC repository.
    Returns:
        tuple[str, bytes]: Filename and file contents as bytes.
    """
    url = dvc.api.get_url(
        path=path,
        repo=repo_folder,
    )
    # Custom S3 endpoint (non-AWS)
    endpoint_url = "https://s3.mesocentre.uca.fr"
    bucket = "mgf-data-products"
    key = url.split(f"{bucket}/")[-1]  # extract key from URL

    # Create S3 client that does NOT require credentials
    s3 = boto3.client(
        "s3",
        endpoint_url=endpoint_url,
        config=Config(signature_version=UNSIGNED),
    )

    # Fetch the object
    obj = s3.get_object(Bucket=bucket, Key=key)

    # Read contents into memory
    data = obj["Body"].read()

    filename = path.split("/")[1] + "_" + path.split("/")[-1]
    # Save to a local file if needed
    with open(filename, "wb") as f:
        f.write(data)

    print("Downloaded", len(data), "bytes from", endpoint_url)
    return path.split("/")[1], data

In [9]:
dvc_fs = DVCFileSystem(rocrate_folder)
files = list(dvc_fs.find("/", detail=False))
print(files)

['/.github/workflows/rocrate_to_pages.yml', '/EMOBON_AAOT_Wa_61-ro-crate.ttl', '/EMOBON_AAOT_Wa_61-ro-crate/.gitignore', '/EMOBON_AAOT_Wa_61-ro-crate/DBB.merged.cmsearch.all.tblout.deoverlapped.bz2', '/EMOBON_AAOT_Wa_61-ro-crate/DBB.merged.fasta.bz2', '/EMOBON_AAOT_Wa_61-ro-crate/DBB.merged.motus.tsv.bz2', '/EMOBON_AAOT_Wa_61-ro-crate/DBB.merged.qc_summary', '/EMOBON_AAOT_Wa_61-ro-crate/DBB.merged.unfiltered_fasta.bz2', '/EMOBON_AAOT_Wa_61-ro-crate/DBB.merged_CDS.faa.bz2', '/EMOBON_AAOT_Wa_61-ro-crate/DBB.merged_CDS.ffn.bz2', '/EMOBON_AAOT_Wa_61-ro-crate/DBB_AAENOSDA_2_1_HVWGWDSX5.UDI183_clean.fastq.trimmed.fasta.bz2', '/EMOBON_AAOT_Wa_61-ro-crate/DBB_AAENOSDA_2_1_HVWGWDSX5.UDI183_clean.fastq.trimmed.qc_summary', '/EMOBON_AAOT_Wa_61-ro-crate/DBB_AAENOSDA_2_2_HVWGWDSX5.UDI183_clean.fastq.trimmed.fasta.bz2', '/EMOBON_AAOT_Wa_61-ro-crate/DBB_AAENOSDA_2_2_HVWGWDSX5.UDI183_clean.fastq.trimmed.qc_summary', '/EMOBON_AAOT_Wa_61-ro-crate/config.yml', '/EMOBON_AAOT_Wa_61-ro-crate/fastp.html', '/

Filter SSU taxonomy tables

In [10]:
filt_files = [f for f in files if f.endswith("SSU-taxonomy-summary.ttl")]
[k for k in filt_files]

['/EMOBON_AAOT_Wa_61-ro-crate/taxonomy-summary/SSU/SSU-taxonomy-summary.ttl',
 '/EMOBON_BPNS_So_17-ro-crate/taxonomy-summary/SSU/SSU-taxonomy-summary.ttl',
 '/EMOBON_BPNS_So_33-ro-crate/taxonomy-summary/SSU/SSU-taxonomy-summary.ttl',
 '/EMOBON_BPNS_So_5-ro-crate/taxonomy-summary/SSU/SSU-taxonomy-summary.ttl',
 '/EMOBON_BPNS_So_6-ro-crate/taxonomy-summary/SSU/SSU-taxonomy-summary.ttl',
 '/EMOBON_BPNS_Wa_46-ro-crate/taxonomy-summary/SSU/SSU-taxonomy-summary.ttl',
 '/EMOBON_NRMCB_So_10-ro-crate/taxonomy-summary/SSU/SSU-taxonomy-summary.ttl',
 '/EMOBON_NRMCB_So_11-ro-crate/taxonomy-summary/SSU/SSU-taxonomy-summary.ttl',
 '/EMOBON_NRMCB_So_12-ro-crate/taxonomy-summary/SSU/SSU-taxonomy-summary.ttl',
 '/EMOBON_NRMCB_So_16-ro-crate/taxonomy-summary/SSU/SSU-taxonomy-summary.ttl',
 '/EMOBON_NRMCB_So_6-ro-crate/taxonomy-summary/SSU/SSU-taxonomy-summary.ttl',
 '/EMOBON_NRMCB_So_7-ro-crate/taxonomy-summary/SSU/SSU-taxonomy-summary.ttl',
 '/EMOBON_NRMCB_Wa_69-ro-crate/taxonomy-summary/SSU/SSU-taxono

### Get SSU tables and upload them to a single graph

In [11]:
dataset = "ssu"
for file in filt_files:
    _, contents = get_single_file_s3(rocrate_folder, file)
    print("Uploading (append):", file)

    create_upload_ds(dataset, contents)

Downloaded 551908 bytes from https://s3.mesocentre.uca.fr
Uploading (append): /EMOBON_AAOT_Wa_61-ro-crate/taxonomy-summary/SSU/SSU-taxonomy-summary.ttl
 551908 bytes from https://s3.mesocentre.uca.fr
Uploading (append): /EMOBON_AAOT_Wa_61-ro-crate/taxonomy-summary/SSU/SSU-taxonomy-summary.ttl
Dataset created: ssu
Upload failed for dataset ssu 400
Server response: [line: 58, col: 9 ] Undefined prefix: dwc

Downloaded 464634 bytes from https://s3.mesocentre.uca.fr
Uploading (append): /EMOBON_BPNS_So_17-ro-crate/taxonomy-summary/SSU/SSU-taxonomy-summary.ttl
Dataset creation failed for ssu 409
Server response: Name already registered '/ssu'

Upload failed for dataset ssu 400
Server response: [line: 58, col: 9 ] Undefined prefix: dwc

Downloaded 481975 bytes from https://s3.mesocentre.uca.fr
Uploading (append): /EMOBON_BPNS_So_33-ro-crate/taxonomy-summary/SSU/SSU-taxonomy-summary.ttl
Dataset creation failed for ssu 409
Server response: Name already registered '/ssu'

Upload failed for datas

## 5. SPARQL filtering
- Now we can demonstrate queries across several graphs
- This is the future added value to organize data in graphs
- Once somebody hosts MGnify data in SPARQL endpoint, you can query all MGnify/metaGOflow data at once

In [12]:
q = """
PREFIX prod:  <https://data.emobon.embrc.eu/ns/product#>
PREFIX dct:   <http://purl.org/dc/terms/>
PREFIX schema:<https://schema.org/>
PREFIX xsd:   <http://www.w3.org/2001/XMLSchema#>

SELECT ?annotation ?otuID ?abundance ?sample ?taxonIRI ?taxonName ?taxonRank
WHERE {
  # annotation node carrying the abundance
  ?annotation a prod:TaxonomicAnnotation ;
              prod:ssuRNA ?abundance ;
              prod:otuID ?otuID ;
              prod:ofSample ?sample ;
              dct:identifier ?taxonIRI .

  # optional taxon metadata reachable from the taxon IRI
  OPTIONAL { ?taxonIRI dct:scientificName ?taxonName }
  OPTIONAL { ?taxonIRI dct:taxonRank ?taxonRank }

  FILTER ( xsd:double(?abundance) > 20 )    # numeric filter
  FILTER ( regex(str(?taxonRank), "^family", "i"))
}
ORDER BY DESC(xsd:double(?abundance))
"""


r = requests.get("http://localhost:3030/ssu", params={"query": q}, headers={"Accept": "application/sparql-results+json"})
df = sparql_json_to_df(r.json())
df

,annotation,otuID,abundance,sample,taxonIRI,taxonName,taxonRank


### Wikipedia query

In [ ]:
q = """
PREFIX wdt:  <http://www.wikidata.org/prop/direct/>
PREFIX rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <https://schema.org/>

SELECT ?item ?label ?wikipediaPage ?ncbi ?mesh 
WHERE {
  SERVICE <https://query.wikidata.org/sparql> {
    ?item wdt:P225 "Flavobacteriaceae" .
    ?item rdfs:label ?label .
    FILTER(LANG(?label) = "en")
    OPTIONAL { ?item wdt:P685 ?ncbi }   # NCBI Taxon ID
    # Find the Wikidata item whose MeSH descriptor ID is the same string
    OPTIONAL { ?item wdt:P672 ?mesh }
  }
}
LIMIT 10
"""

r = requests.get("http://localhost:3030/ssu", params={"query": q}, headers={"Accept": "application/sparql-results+json"})
df = sparql_json_to_df(r.json())
df

,item,label,wikipediaPage,ncbi,mesh
0,http://www.wikidata.org/entity/Q5458145,Flavobacteriaceae,None,49546,B03.440.080.190
1,http://www.wikidata.org/entity/Q5458145,Flavobacteriaceae,None,49546,B03.440.400.425.310


### UniProt
- example queries, https://sparql.uniprot.org/.well-known/sparql-examples/

In [16]:
q = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX taxon: <http://purl.uniprot.org/taxonomy/>
PREFIX up: <http://purl.uniprot.org/core/>

SELECT ?protein ?organism ?sequence
WHERE {
    ?protein a up:Protein ;
             up:organism ?organism ;
             up:sequence ?seqNode .
    ?seqNode rdf:value ?sequence .
    
    # Only proteins under taxon 49546
    ?organism rdfs:subClassOf taxon:49546 .
}
LIMIT 100
""" 

r = requests.get("https://sparql.uniprot.org/sparql", params={"query": q}, headers={"Accept": "application/sparql-results+json"})
df = sparql_json_to_df(r.json())
df

,protein,organism,sequence
0,http://purl.uniprot.org/uniprot/A0A023BMI4,http://purl.uniprot.org/taxonomy/1317122,MYSRTHFSELLSLIPRYKFNQFVLKYSADKHNKGFNSWTHLVTMVFSQLSKANSLREIETSFNSVVNAHFHMGARSIKRSTLSEANQKRDFRVFADLANELMKNFRPSKQKELKEFLFLLDSSPIILQGRHFDWTNKTRNYNNGLKLHMLYDTHTTTPTYIDITASNINDINIGRELPIQPNATYVFDKGYTDYNWWFSIHKKQSFFVTRFKKNAATHIIEELPINKSDTQLVLADQKVIFKNKTPRGGKINQYTVPLRKITIRRDNKNTPLVIATNDFNKSAGEIASLYKKRWDIELFFKWIKQNLKIKRFIGTSLNAVKTQIYTAIITYLLSLKLQKLKENTLPFYLFLEKLSALLFVPVTLIKNDGHSQKKKDQLLIKQQLNFSW
1,http://purl.uniprot.org/uniprot/A0A023BMI7,http://purl.uniprot.org/taxonomy/1317122,MDFTIRKGSHTVSRLTCHIVWSTKYRYKVLRGDIQIRCRELLIQICDAEGIEILKGVVSADHVHMHIEYAPKLSVSYVVKQLKGRTSRKLQQEFHSLQDRYWGKHFWANGYGVWSTGNITDKMVNEYLEHHRRDNNDNSNFILE
2,http://purl.uniprot.org/uniprot/A0A023BMJ1,http://purl.uniprot.org/taxonomy/1317122,MKLPKTYIKIPITELGELRFMQDTLLDNLSLLSQTEDNFSTNRSIKDNMYWISKILVAIAEIDQRREFDDLELEKK
3,http://purl.uniprot.org/uniprot/A0A023BMJ3,http://purl.uniprot.org/taxonomy/1317122,MKKIIFVVFFLMTYIGFSQDYGFLFKTEVQATAHANLFNASVVTDSPEVYPGIGYNSTPQNVGSIYYDFIELNNNFNTATVNLYSHWISLSDPVSCNSDDTYTYTRNEFINNLVGYNTKDCNFFTIVYPIHIIEPSANEFCPDQEIVLKYGYHWQFSFDGINWNSFPTSLNTKRVTSFTLKELFSLSGIPDSQWQSESNIKFQTGYRTEFTNIRNITIINCSPKLDGPIIDIQPLCSNSINHNDNDNGSFTVTFDRELDDTKQEKMNLQVYRQVGSSFDGYASKVVTKSDFTGTSYTWEPKNLPGGVYKLFWQTKSNNEGFDDINTVPDAYDESNPFTLTTPPALSVSGAPSPVQCFGGNDGSITVTPNGGTPGTPPTSPRYQYSIDNGTTWQQETLFDSLTKGDYTILIKDNNGCEATSAPITVNERFLTIPDVVGLSALITSPTLINGNNGRIAISVSSGSGNYTNYAWTKDGNPFTPPSGSTNTNIINLYEGVYTIVVTDSNGCSSNLETFTLTDPEPIDISINMTPNTVNCSDTKVNLIASATGGFLNSGGDYTYLWDDGTTEASLTNVGIGNYQVTVSDQGGNSQSKSFQVQGPEPITAIPTVSNVGCKNGSDGTIQLTINGGTGQYTVNWTKLFDNT
4,http://purl.uniprot.org/uniprot/A0A023BMJ6,http://purl.uniprot.org/taxonomy/1317122,MIDQGLNFNIFNIIILIGIFQGPIFALIVFFNKNYRFLANYFLVSTALALSFNNFQYWLLDTGMVNELYFQIPFEFLIMSMFYPFVDEYLQIKSPKKIILAIIVPFFTSFIFRLIMKFGLITLSNDLIHILLTLEEYLSLVFSVSMITIILIKIHKYEKAKTDFNLSEIKAKTKWLKQALVFGIIICVFWVFVIQDNIARFEDDLSKYYPLWIIISILVYWIVYKGIIETQIFNQRIEIRNDTIEFTYNGQKTAYINDDFFLEIKSFIINEKLYLNPNLNLDLVAEKFNVSIGHLSKTVNKNANQSFTDFVNQLRVNESKKMLLNPNYKNYTIEAIGYESGFYSKSNFYAAFKKETNQTPSAFRLRK
...,...,...,...
95,http://purl.uniprot.org/uniprot/A0A023BMZ8,http://purl.uniprot.org/taxonomy/1317122,MKFENLIICVLTGFVVVSGYTQEKTIDTTLVNELQEVVLTATRTERQLSSLPLPVTIVSQETIKQSGTIRLNEILNEQTGIITVADESGFEGVQIQGIASDYILILIDGVPLVGRKAGNFDVNRLTVGNIKQVEVVKGPSSSLYGSEALGGVINIITEKPKSDVLSGNASYRIGSYTQQDINVDIKQRIKKLGYGVFANRFSSEGYDLTPDTAGQTVNPFENYTFNGRLYYDFSDQFSLFLSGRLYTQYQDAGFTTNTTSFEGDSEEKEWNSHLRLDHKWSDHLTTQYEFYYTNYNAKEQLADSSSGDIVSDSDFDQRLLRPEIRTTYAFKDSSKLTFGVGFQYDELDRTFFDKQVDFNSQYVYAQYDTHLIERLNVITGARFDNHSEYSNQFSPKLALRYKITEALAAKASVGYGFKAPDFRQLYFDFTNSTVGYTVLGYNVALEKLNELQAQGQILDVVVPESSLQDPLEAENSIGYNAGLTYKENRWNAELNFFRNDFKNLIDTRVIARKTNGQNVFSYFNFDKIYTTGLEFNTNYRITDNVRLSAGYQLLYAFDKEKERLVKNGEVFARDPETNQTVAVSRSEYFGLVNRSRHNANFKVFYDIVSAKANINLRLLYRSKYALFDTNGNDLIDDYDTSFVDGFAIANIAASKTFYENFTLQIGANNLFDYTKDNIPTLPGIQLYAKLNYQF
96,http://purl.uniprot.org/uniprot/A0A023BMZ9,http://purl.uniprot.org/taxonomy/1317122,MKKKHFLLISVSLLMSQGLLAQDHSTHSSPGSLGAEQIFGLLEMPFLAIALIFSFLTATKLKGGKFGSGMTLLAWGFVVMALGHLHMQIAHIFDYNIFKNIFGDTFGNYIWFIALILTWGLSALGFYKIYKASKI
97,http://purl.uniprot.org/uniprot/A0A023BN00,http://purl.uniprot.org/taxonomy/1317122,MKVNVHQNIKILVLVLSILFSLQKSYSQNDNFWSNVSFGGNLGIGFGNDTFSGVIEPSALYNFNEQFAAGMGVSFGYIESNNFTATNYGGSLLAFYSPIREIRLSLEFQEMGVSRTLEIENAQDLKENYWYPSLFVGGGYRMGNVSVGIRYDLLYDSDKSIYGSAYTPFVSVFF
98,http://purl.uniprot.org/uniprot/A0A023BN01,http://purl.uniprot.org/taxonomy/1317122,MTPVVSGFSKLSKADKIKWLAKHHFNDDQNAVDTLVTYWNSDDGLQQLHDEFTENTISNYYLPFSVAPNFLINNKRYTLPMAIEESSVVAAASKAAKFWQTRGGFKAEVLSTIKVGQVHFTYNGKPEKLQQFFSIIKPKLLASVSHMTKNMEKRGGGVIDIELRDKTSEIDDYYQLHCTFETVDAMGANFINSCLEQFAKTMTTEAKEHHDFSATEKDIEIVMSILSNYVPQCLVKASVSCNIKDLPSSPSLSPLQYANKFVRAVRIAEVEPYRAVTHNKGIMNGIDAVVLATGNDFRAIEAGAHAYASRDGKYTSLTHAEIQNEMLTFSIKLPLALGTVGGLTSLHPLVKFALQLLEKPNAKKLMEITAVAGLAQNFAAINSLITTGIQQGHMKMHLMNILNQFKATENEKKQLIKYFETNAVTHSEVVTQIEKLRA


In [17]:
q = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX taxon: <http://purl.uniprot.org/taxonomy/>
PREFIX up: <http://purl.uniprot.org/core/>

SELECT (COUNT(?protein) AS ?proteinCount)
WHERE {
    ?protein a up:Protein ;
             up:organism ?organism ;
             up:sequence ?seqNode .
    ?seqNode rdf:value ?sequence .

    ?organism rdfs:subClassOf taxon:49546 .
}

"""
r = requests.get("https://sparql.uniprot.org/sparql", params={"query": q}, headers={"Accept": "application/sparql-results+json"})
df = sparql_json_to_df(r.json())
df

,proteinCount
0,2829119


## Total query

In [18]:
q = """
PREFIX prod:  <https://data.emobon.embrc.eu/ns/product#>
PREFIX dct:   <http://purl.org/dc/terms/>
PREFIX schema:<https://schema.org/>
PREFIX xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX wdt:  <http://www.wikidata.org/prop/direct/>
PREFIX rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX taxon: <http://purl.uniprot.org/taxonomy/>
PREFIX up: <http://purl.uniprot.org/core/>

SELECT ?annotation ?otuID ?abundance ?sample ?taxonIRI ?taxonTitle ?taxonName ?taxonRank ?item ?label ?wikipediaPage ?ncbi ?mesh
WHERE {
  {
    SELECT ?annotation ?otuID ?abundance ?sample ?taxonIRI ?taxonTitle ?taxonName ?taxonRank
    WHERE {
      ?annotation a prod:TaxonomicAnnotation ;
                  prod:ssuRNA ?abundance ;
                  prod:otuID ?otuID ;
                  prod:ofSample ?sample ;
                  dct:identifier ?taxonIRI .
      OPTIONAL { ?taxonIRI dct:title ?taxonTitle }
      OPTIONAL { ?taxonIRI dct:scientificName ?taxonName }
      OPTIONAL { ?taxonIRI dct:taxonRank ?taxonRank }

      FILTER(xsd:double(?abundance) > 20)
      FILTER(regex(str(?taxonRank), "^family", "i"))
    }
    ORDER BY DESC(xsd:double(?abundance))
    LIMIT 1
  }

  SERVICE <https://query.wikidata.org/sparql> {
    ?item wdt:P225 ?taxonName .
    ?item rdfs:label ?label .
    FILTER(LANG(?label) = "en")
    OPTIONAL { ?item wdt:P685 ?ncbi }   # NCBI Taxon ID
    # Find the Wikidata item whose MeSH descriptor ID is the same string
    OPTIONAL { ?item wdt:P672 ?mesh }
  }
}
"""


r = requests.get("http://localhost:3030/ssu", params={"query": q}, headers={"Accept": "application/sparql-results+json"})
df = sparql_json_to_df(r.json())
df

,annotation,otuID,abundance,sample,taxonIRI,taxonTitle,taxonName,taxonRank,item,label,wikipediaPage,ncbi,mesh
0,https://data.emobon.embrc.eu/analysis-results-cluster-01-crate/EMOBON_HCMR-1_Wa_6-ro-crate/taxonomy-summary-SSU#49546,23045,2121.0,http://data.emobon.embrc.eu/observatory-hcmr-1-crate/water/sample/EMOBON_HCMR-1_Wa_6,https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=49546,Flavobacteriaceae,Flavobacteriaceae,family,http://www.wikidata.org/entity/Q5458145,Flavobacteriaceae,None,49546,B03.440.080.190
1,https://data.emobon.embrc.eu/analysis-results-cluster-01-crate/EMOBON_HCMR-1_Wa_6-ro-crate/taxonomy-summary-SSU#49546,23045,2121.0,http://data.emobon.embrc.eu/observatory-hcmr-1-crate/water/sample/EMOBON_HCMR-1_Wa_6,https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=49546,Flavobacteriaceae,Flavobacteriaceae,family,http://www.wikidata.org/entity/Q5458145,Flavobacteriaceae,None,49546,B03.440.400.425.310


In [19]:
tax_id = df.loc[0, "ncbi"]
q = f"""
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX taxon: <http://purl.uniprot.org/taxonomy/>
PREFIX up: <http://purl.uniprot.org/core/>

SELECT ?taxon ?protein ?organism ?sequence
WHERE {{
    ?protein a up:Protein ;
             up:organism ?organism ;
             up:sequence ?seqNode .
    ?seqNode rdf:value ?sequence .
    
    # Only proteins under taxon
    ?organism rdfs:subClassOf taxon:{tax_id} .
    BIND({tax_id} AS ?taxon)
}}
LIMIT 100
""" 

r = requests.get("https://sparql.uniprot.org/sparql", params={"query": q}, headers={"Accept": "application/sparql-results+json"})
df_prot = sparql_json_to_df(r.json())
df_prot

,taxon,protein,organism,sequence
0,49546,http://purl.uniprot.org/uniprot/A0A023BMI4,http://purl.uniprot.org/taxonomy/1317122,MYSRTHFSELLSLIPRYKFNQFVLKYSADKHNKGFNSWTHLVTMVFSQLSKANSLREIETSFNSVVNAHFHMGARSIKRSTLSEANQKRDFRVFADLANELMKNFRPSKQKELKEFLFLLDSSPIILQGRHFDWTNKTRNYNNGLKLHMLYDTHTTTPTYIDITASNINDINIGRELPIQPNATYVFDKGYTDYNWWFSIHKKQSFFVTRFKKNAATHIIEELPINKSDTQLVLADQKVIFKNKTPRGGKINQYTVPLRKITIRRDNKNTPLVIATNDFNKSAGEIASLYKKRWDIELFFKWIKQNLKIKRFIGTSLNAVKTQIYTAIITYLLSLKLQKLKENTLPFYLFLEKLSALLFVPVTLIKNDGHSQKKKDQLLIKQQLNFSW
1,49546,http://purl.uniprot.org/uniprot/A0A023BMI7,http://purl.uniprot.org/taxonomy/1317122,MDFTIRKGSHTVSRLTCHIVWSTKYRYKVLRGDIQIRCRELLIQICDAEGIEILKGVVSADHVHMHIEYAPKLSVSYVVKQLKGRTSRKLQQEFHSLQDRYWGKHFWANGYGVWSTGNITDKMVNEYLEHHRRDNNDNSNFILE
2,49546,http://purl.uniprot.org/uniprot/A0A023BMJ1,http://purl.uniprot.org/taxonomy/1317122,MKLPKTYIKIPITELGELRFMQDTLLDNLSLLSQTEDNFSTNRSIKDNMYWISKILVAIAEIDQRREFDDLELEKK
3,49546,http://purl.uniprot.org/uniprot/A0A023BMJ3,http://purl.uniprot.org/taxonomy/1317122,MKKIIFVVFFLMTYIGFSQDYGFLFKTEVQATAHANLFNASVVTDSPEVYPGIGYNSTPQNVGSIYYDFIELNNNFNTATVNLYSHWISLSDPVSCNSDDTYTYTRNEFINNLVGYNTKDCNFFTIVYPIHIIEPSANEFCPDQEIVLKYGYHWQFSFDGINWNSFPTSLNTKRVTSFTLKELFSLSGIPDSQWQSESNIKFQTGYRTEFTNIRNITIINCSPKLDGPIIDIQPLCSNSINHNDNDNGSFTVTFDRELDDTKQEKMNLQVYRQVGSSFDGYASKVVTKSDFTGTSYTWEPKNLPGGVYKLFWQTKSNNEGFDDINTVPDAYDESNPFTLTTPPALSVSGAPSPVQCFGGNDGSITVTPNGGTPGTPPTSPRYQYSIDNGTTWQQETLFDSLTKGDYTILIKDNNGCEATSAPITVNERFLTIPDVVGLSALITSPTLINGNNGRIAISVSSGSGNYTNYAWTKDGNPFTPPSGSTNTNIINLYEGVYTIVVTDSNGCSSNLETFTLTDPEPIDISINMTPNTVNCSDTKVNLIASATGGFLNSGGDYTYLWDDGTTEASLTNVGIGNYQVTVSDQGGNSQSKSFQVQGPEPITAIPTVSNVGCKNGSDGTIQLTINGGTGQYTVNWTKLFDNT
4,49546,http://purl.uniprot.org/uniprot/A0A023BMJ6,http://purl.uniprot.org/taxonomy/1317122,MIDQGLNFNIFNIIILIGIFQGPIFALIVFFNKNYRFLANYFLVSTALALSFNNFQYWLLDTGMVNELYFQIPFEFLIMSMFYPFVDEYLQIKSPKKIILAIIVPFFTSFIFRLIMKFGLITLSNDLIHILLTLEEYLSLVFSVSMITIILIKIHKYEKAKTDFNLSEIKAKTKWLKQALVFGIIICVFWVFVIQDNIARFEDDLSKYYPLWIIISILVYWIVYKGIIETQIFNQRIEIRNDTIEFTYNGQKTAYINDDFFLEIKSFIINEKLYLNPNLNLDLVAEKFNVSIGHLSKTVNKNANQSFTDFVNQLRVNESKKMLLNPNYKNYTIEAIGYESGFYSKSNFYAAFKKETNQTPSAFRLRK
...,...,...,...,...
95,49546,http://purl.uniprot.org/uniprot/A0A023BMZ8,http://purl.uniprot.org/taxonomy/1317122,MKFENLIICVLTGFVVVSGYTQEKTIDTTLVNELQEVVLTATRTERQLSSLPLPVTIVSQETIKQSGTIRLNEILNEQTGIITVADESGFEGVQIQGIASDYILILIDGVPLVGRKAGNFDVNRLTVGNIKQVEVVKGPSSSLYGSEALGGVINIITEKPKSDVLSGNASYRIGSYTQQDINVDIKQRIKKLGYGVFANRFSSEGYDLTPDTAGQTVNPFENYTFNGRLYYDFSDQFSLFLSGRLYTQYQDAGFTTNTTSFEGDSEEKEWNSHLRLDHKWSDHLTTQYEFYYTNYNAKEQLADSSSGDIVSDSDFDQRLLRPEIRTTYAFKDSSKLTFGVGFQYDELDRTFFDKQVDFNSQYVYAQYDTHLIERLNVITGARFDNHSEYSNQFSPKLALRYKITEALAAKASVGYGFKAPDFRQLYFDFTNSTVGYTVLGYNVALEKLNELQAQGQILDVVVPESSLQDPLEAENSIGYNAGLTYKENRWNAELNFFRNDFKNLIDTRVIARKTNGQNVFSYFNFDKIYTTGLEFNTNYRITDNVRLSAGYQLLYAFDKEKERLVKNGEVFARDPETNQTVAVSRSEYFGLVNRSRHNANFKVFYDIVSAKANINLRLLYRSKYALFDTNGNDLIDDYDTSFVDGFAIANIAASKTFYENFTLQIGANNLFDYTKDNIPTLPGIQLYAKLNYQF
96,49546,http://purl.uniprot.org/uniprot/A0A023BMZ9,http://purl.uniprot.org/taxonomy/1317122,MKKKHFLLISVSLLMSQGLLAQDHSTHSSPGSLGAEQIFGLLEMPFLAIALIFSFLTATKLKGGKFGSGMTLLAWGFVVMALGHLHMQIAHIFDYNIFKNIFGDTFGNYIWFIALILTWGLSALGFYKIYKASKI
97,49546,http://purl.uniprot.org/uniprot/A0A023BN00,http://purl.uniprot.org/taxonomy/1317122,MKVNVHQNIKILVLVLSILFSLQKSYSQNDNFWSNVSFGGNLGIGFGNDTFSGVIEPSALYNFNEQFAAGMGVSFGYIESNNFTATNYGGSLLAFYSPIREIRLSLEFQEMGVSRTLEIENAQDLKENYWYPSLFVGGGYRMGNVSVGIRYDLLYDSDKSIYGSAYTPFVSVFF
98,49546,http://purl.uniprot.org/uniprot/A0A023BN01,http://purl.uniprot.org/taxonomy/1317122,MTPVVSGFSKLSKADKIKWLAKHHFNDDQNAVDTLVTYWNSDDGLQQLHDEFTENTISNYYLPFSVAPNFLINNKRYTLPMAIEESSVVAAASKAAKFWQTRGGFKAEVLSTIKVGQVHFTYNGKPEKLQQFFSIIKPKLLASVSHMTKNMEKRGGGVIDIELRDKTSEIDDYYQLHCTFETVDAMGANFINSCLEQFAKTMTTEAKEHHDFSATEKDIEIVMSILSNYVPQCLVKASVSCNIKDLPSSPSLSPLQYANKFVRAVRIAEVEPYRAVTHNKGIMNGIDAVVLATGNDFRAIEAGAHAYASRDGKYTSLTHAEIQNEMLTFSIKLPLALGTVGGLTSLHPLVKFALQLLEKPNAKKLMEITAVAGLAQNFAAINSLITTGIQQGHMKMHLMNILNQFKATENEKKQLIKYFETNAVTHSEVVTQIEKLRA


## Final DF merge, data from local EMO-BON + wikipedia + UniProt!!!

In [20]:
# merge dataframes
df_merged = df.merge(df_prot, how="left", left_on="ncbi", right_on="taxon")
df_merged

,annotation,otuID,abundance,sample,taxonIRI,taxonTitle,taxonName,taxonRank,item,label,wikipediaPage,ncbi,mesh,taxon,protein,organism,sequence
0,https://data.emobon.embrc.eu/analysis-results-cluster-01-crate/EMOBON_HCMR-1_Wa_6-ro-crate/taxonomy-summary-SSU#49546,23045,2121.0,http://data.emobon.embrc.eu/observatory-hcmr-1-crate/water/sample/EMOBON_HCMR-1_Wa_6,https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=49546,Flavobacteriaceae,Flavobacteriaceae,family,http://www.wikidata.org/entity/Q5458145,Flavobacteriaceae,None,49546,B03.440.080.190,49546,http://purl.uniprot.org/uniprot/A0A023BMI4,http://purl.uniprot.org/taxonomy/1317122,MYSRTHFSELLSLIPRYKFNQFVLKYSADKHNKGFNSWTHLVTMVFSQLSKANSLREIETSFNSVVNAHFHMGARSIKRSTLSEANQKRDFRVFADLANELMKNFRPSKQKELKEFLFLLDSSPIILQGRHFDWTNKTRNYNNGLKLHMLYDTHTTTPTYIDITASNINDINIGRELPIQPNATYVFDKGYTDYNWWFSIHKKQSFFVTRFKKNAATHIIEELPINKSDTQLVLADQKVIFKNKTPRGGKINQYTVPLRKITIRRDNKNTPLVIATNDFNKSAGEIASLYKKRWDIELFFKWIKQNLKIKRFIGTSLNAVKTQIYTAIITYLLSLKLQKLKENTLPFYLFLEKLSALLFVPVTLIKNDGHSQKKKDQLLIKQQLNFSW
1,https://data.emobon.embrc.eu/analysis-results-cluster-01-crate/EMOBON_HCMR-1_Wa_6-ro-crate/taxonomy-summary-SSU#49546,23045,2121.0,http://data.emobon.embrc.eu/observatory-hcmr-1-crate/water/sample/EMOBON_HCMR-1_Wa_6,https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=49546,Flavobacteriaceae,Flavobacteriaceae,family,http://www.wikidata.org/entity/Q5458145,Flavobacteriaceae,None,49546,B03.440.080.190,49546,http://purl.uniprot.org/uniprot/A0A023BMI7,http://purl.uniprot.org/taxonomy/1317122,MDFTIRKGSHTVSRLTCHIVWSTKYRYKVLRGDIQIRCRELLIQICDAEGIEILKGVVSADHVHMHIEYAPKLSVSYVVKQLKGRTSRKLQQEFHSLQDRYWGKHFWANGYGVWSTGNITDKMVNEYLEHHRRDNNDNSNFILE
2,https://data.emobon.embrc.eu/analysis-results-cluster-01-crate/EMOBON_HCMR-1_Wa_6-ro-crate/taxonomy-summary-SSU#49546,23045,2121.0,http://data.emobon.embrc.eu/observatory-hcmr-1-crate/water/sample/EMOBON_HCMR-1_Wa_6,https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=49546,Flavobacteriaceae,Flavobacteriaceae,family,http://www.wikidata.org/entity/Q5458145,Flavobacteriaceae,None,49546,B03.440.080.190,49546,http://purl.uniprot.org/uniprot/A0A023BMJ1,http://purl.uniprot.org/taxonomy/1317122,MKLPKTYIKIPITELGELRFMQDTLLDNLSLLSQTEDNFSTNRSIKDNMYWISKILVAIAEIDQRREFDDLELEKK
3,https://data.emobon.embrc.eu/analysis-results-cluster-01-crate/EMOBON_HCMR-1_Wa_6-ro-crate/taxonomy-summary-SSU#49546,23045,2121.0,http://data.emobon.embrc.eu/observatory-hcmr-1-crate/water/sample/EMOBON_HCMR-1_Wa_6,https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=49546,Flavobacteriaceae,Flavobacteriaceae,family,http://www.wikidata.org/entity/Q5458145,Flavobacteriaceae,None,49546,B03.440.080.190,49546,http://purl.uniprot.org/uniprot/A0A023BMJ3,http://purl.uniprot.org/taxonomy/1317122,MKKIIFVVFFLMTYIGFSQDYGFLFKTEVQATAHANLFNASVVTDSPEVYPGIGYNSTPQNVGSIYYDFIELNNNFNTATVNLYSHWISLSDPVSCNSDDTYTYTRNEFINNLVGYNTKDCNFFTIVYPIHIIEPSANEFCPDQEIVLKYGYHWQFSFDGINWNSFPTSLNTKRVTSFTLKELFSLSGIPDSQWQSESNIKFQTGYRTEFTNIRNITIINCSPKLDGPIIDIQPLCSNSINHNDNDNGSFTVTFDRELDDTKQEKMNLQVYRQVGSSFDGYASKVVTKSDFTGTSYTWEPKNLPGGVYKLFWQTKSNNEGFDDINTVPDAYDESNPFTLTTPPALSVSGAPSPVQCFGGNDGSITVTPNGGTPGTPPTSPRYQYSIDNGTTWQQETLFDSLTKGDYTILIKDNNGCEATSAPITVNERFLTIPDVVGLSALITSPTLINGNNGRIAISVSSGSGNYTNYAWTKDGNPFTPPSGSTNTNIINLYEGVYTIVVTDSNGCSSNLETFTLTDPEPIDISINMTPNTVNCSDTKVNLIASATGGFLNSGGDYTYLWDDGTTEASLTNVGIGNYQVTVSDQGGNSQSKSFQVQGPEPITAIPTVSNVGCKNGSDGTIQLTINGGTGQYTVNWTKLFDNT
4,https://data.emobon.embrc.eu/analysis-results-cluster-01-crate/EMOBON_HCMR-1_Wa_6-ro-crate/taxonomy-summary-SSU#49546,23045,2121.0,http://data.emobon.embrc.eu/observatory-hcmr-1-crate/water/sample/EMOBON_HCMR-1_Wa_6,https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=49546,Flavobacteriaceae,Flavobacteriaceae,family,http://www.wikidata.org/entity/Q5458145,Flavobacteriaceae,None,49546,B03.440.080.190,49546,http://purl.uniprot.org/uniprot/A0A023BMJ6,http://purl.uniprot.org/taxonomy/1317122,MIDQGLNFNIFNIIILIGIFQGPIFALIVFFNKNYRFLANYFLVSTALALSFNNFQYWLLDTGMVNELYFQIPFEFLIMSMFYPFVDEYLQIK